In [1]:
import numpy as np
import matplotlib.pyplot as plt
import data_generation as gen
import torch

## Synthetics generation

In [ ]:
signal, source, stations = gen.signal.generate_one_signal(plot=True)

#### TauP comparison

In [ ]:
# Source characteristics
source_lat = source[0]
source_lon = source[1]
source_depth = source[2]
print(f'Depth : {source_depth/1e3:.3f} km')

# Station position
station_lat = stations[0][0]
station_lon = stations[0][1]

In [ ]:
# My solution
P, pP, sP = gen.arrival_time.travel_times(source_lat, source_lon, source_depth, station_lat, station_lon)
tpP = pP - P
tsP = sP - P

# Results
print(f'P-pP: {tpP:.3f}s - P-sP: {tsP:.3f}s')

In [ ]:
# TauP solution
from obspy.taup import TauPyModel
from obspy.geodetics import locations2degrees

# Kenett model AK135
model = TauPyModel(model="ak135")

distance_deg = locations2degrees(lat1=source_lat, long1=source_lon,
                                  lat2=station_lat, long2=station_lon)

arrivals = model.get_travel_times(source_depth_in_km=source_depth/1e3,
                                  distance_in_degree=distance_deg,
                                  phase_list=["P", "pP", "sP"])

# Results
tpP = arrivals[1].time - arrivals[0].time
tsP = arrivals[2].time - arrivals[0].time

print(f'P-pP: {tpP:.3f}s - P-sP: {tsP:.3f}s')

#### Generate matrix and plot

In [ ]:
signal_matrix, depth = gen.matrix.generate_matrix()

# Plot
plt.figure(figsize=(10,7))
plt.imshow(signal_matrix, aspect='auto', cmap='viridis', origin='upper')

# Adjust x-axis to represent time in seconds
num_columns = len(signal_matrix[0])  # Number of columns in the matrix
plt.xticks(
    ticks=np.arange(0, num_columns, step=1000),  # Adjust step size as needed
    labels=np.arange(0, num_columns / 100, step=1000 / 100)  # Convert to seconds (1/100 of a second since 100 Hz sampling)
)

#plt.colorbar(label='Amplitude')
plt.xlabel('Time (s)')
plt.ylabel('Signals (organized by distance)')
plt.title(f'Energetic envelope of the Z-normalized signals aligned with P-arrival \nDepth : {depth/1e3:.1f} km', fontweight='bold')
plt.tight_layout()
plt.show()

## CNN-DNN model

In [2]:
model, train_data, test_data = gen.depth_model.run_DepthModel()

Succesfully generated train dataset.
Succesfully generated test dataset.
Succesfully initialized model using cuda.
Succesfully set loss criterion and optimizer.


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.58 GiB. GPU 0 has a total capacity of 12.00 GiB of which 0 bytes is free. Of the allocated memory 18.35 GiB is allocated by PyTorch, and 280.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)